In [131]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [132]:
submission_data=pd.read_csv('https://stepik.org/media/attachments/course/4852/submissions_data_train.zip')
submission_data_test=pd.read_csv('https://stepik.org/media/attachments/course/4852/submission_data_test.csv')

In [133]:
events_data_test=pd.read_csv('https://stepik.org/media/attachments/course/4852/events_data_test.csv')
events_data=pd.read_csv('https://stepik.org/media/attachments/course/4852/event_data_train.zip')

In [134]:
submission_data

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853
...,...,...,...,...
509099,120745,1501946959,wrong,2615
509100,120745,1503059927,correct,13177
509101,120745,1506348153,wrong,15253
509102,120745,1523035316,wrong,1424


In [135]:
events_data

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632
...,...,...,...,...
3480698,33375,1526772685,viewed,11677
3480699,32815,1526772750,viewed,5198
3480700,32815,1526772750,discovered,5198
3480701,32815,1526772750,passed,5198


In [136]:
X_train=pd.DataFrame({'user_id':events_data['user_id'].unique()})

In [137]:
cor=submission_data[submission_data['submission_status']=='correct'].groupby('user_id').agg({'step_id':'nunique'}).reset_index().rename({'step_id':'n_correct'},axis=1)

In [138]:
X_train=X_train.merge(cor[['user_id','n_correct']],how='outer').fillna(0)

In [139]:
X_train['pass_c']=X_train.n_correct>=40

In [140]:
X_train

,user_id,n_correct,pass_c
0,17632,30.0,False
1,12494,17.0,False
2,442,25.0,False
3,22254,24.0,False
4,6646,70.0,True
...,...,...,...
19229,795,2.0,False
19230,13475,2.0,False
19231,22851,0.0,False
19232,943,1.0,False


In [141]:
users_drop=events_data.groupby('user_id',as_index=False).agg({'timestamp':'max'})

In [142]:
dropout=30*24*60*60

In [143]:
now=1526772811

In [144]:
users_drop['dropped']=(now-users_drop.timestamp)>=dropout

In [145]:
X_train=X_train.merge(users_drop[['user_id','dropped']],how='outer')

In [146]:
submission_data_train=submission_data[events_data.groupby('user_id')['timestamp'].apply(lambda x:x<=(x.min()+2*24*60*60))]

C:\Users\User\AppData\Local\Temp/ipykernel_4928/763559960.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  submission_data_train=submission_data[events_data.groupby('user_id')['timestamp'].apply(lambda x:x<=(x.min()+2*24*60*60))]


In [147]:
events_data_train=events_data[events_data.groupby('user_id')['timestamp'].apply(lambda x:x<=(x.min()+2*24*60*60))]

In [148]:
steps_tried=submission_data_train.groupby('user_id').step_id.nunique().to_frame().reset_index().rename({'step_id':"steps_tried"},axis=1)

In [149]:
steps_tried

,user_id,steps_tried
0,3,10
1,16,37
2,22,6
3,26,2
4,30,17
...,...,...
5442,26779,10
5443,26781,15
5444,26784,4
5445,26786,10


In [150]:
X_train=X_train.merge(steps_tried[['user_id','steps_tried']],how='outer').fillna(0)

In [151]:
X_train=X_train.drop('n_correct',axis=1)

In [152]:
X_train=X_train.merge(steps_tried[['user_id','steps_tried']],how='outer').fillna(0)

In [153]:
X_train

,user_id,pass_c,dropped,steps_tried
0,17632,False,True,43.0
1,12494,False,True,15.0
2,442,False,True,28.0
3,22254,False,True,24.0
4,6646,True,True,16.0
...,...,...,...,...
19229,795,False,False,0.0
19230,13475,False,False,0.0
19231,22851,False,False,0.0
19232,943,False,False,0.0


In [154]:
users_scores=submission_data_train.pivot_table(index="user_id",columns="submission_status",values="step_id",aggfunc="count",fill_value=0).reset_index()

In [155]:
X_train=X_train.merge(users_scores[['user_id','correct','wrong']],how='outer').fillna(0)

In [156]:
X_train['cor_score']=X_train.correct/(X_train.correct+X_train.wrong)

In [157]:
X_train=X_train.fillna(0)

In [158]:
X_train

,user_id,pass_c,dropped,steps_tried,correct,wrong,cor_score
0,17632,False,True,43.0,31.0,278.0,0.100324
1,12494,False,True,15.0,14.0,36.0,0.280000
2,442,False,True,28.0,25.0,134.0,0.157233
3,22254,False,True,24.0,24.0,54.0,0.307692
4,6646,True,True,16.0,13.0,4.0,0.764706
...,...,...,...,...,...,...,...
19229,795,False,False,0.0,0.0,0.0,0.000000
19230,13475,False,False,0.0,0.0,0.0,0.000000
19231,22851,False,False,0.0,0.0,0.0,0.000000
19232,943,False,False,0.0,0.0,0.0,0.000000


In [159]:
users_events=events_data_train.pivot_table(index="user_id",columns="action",values="step_id",aggfunc="count",fill_value=0).reset_index()

In [160]:
users_events

action,user_id,discovered,passed,started_attempt,viewed
0,1,1,0,0,1
1,2,9,9,2,9
2,3,15,15,4,20
3,5,1,1,0,1
4,7,1,1,0,1
...,...,...,...,...,...
19229,26790,2,2,0,2
19230,26793,1,0,1,1
19231,26794,50,50,24,90
19232,26797,10,10,2,10


In [161]:
X_train=X_train.merge(users_events[['user_id','viewed']])

In [162]:
X_train=X_train[~((X_train.dropped==False) & (X_train.pass_c==False))]

In [163]:
X_train

,user_id,pass_c,dropped,steps_tried,correct,wrong,cor_score,viewed
0,17632,False,True,43.0,31.0,278.0,0.100324,29
1,12494,False,True,15.0,14.0,36.0,0.280000,24
2,442,False,True,28.0,25.0,134.0,0.157233,31
3,22254,False,True,24.0,24.0,54.0,0.307692,36
4,6646,True,True,16.0,13.0,4.0,0.764706,3
...,...,...,...,...,...,...,...,...
18916,18148,True,False,1.0,1.0,0.0,1.000000,116
18925,12745,True,False,0.0,0.0,0.0,0.000000,41
18959,193,True,False,1.0,0.0,2.0,0.000000,55
18971,15597,True,False,6.0,3.0,6.0,0.333333,283


In [164]:
y_train=X_train.pass_c

In [165]:
X_train=X_train.drop(['user_id','pass_c','dropped'],axis=1)

In [166]:
X_train

,steps_tried,correct,wrong,cor_score,viewed
0,43.0,31.0,278.0,0.100324,29
1,15.0,14.0,36.0,0.280000,24
2,28.0,25.0,134.0,0.157233,31
3,24.0,24.0,54.0,0.307692,36
4,16.0,13.0,4.0,0.764706,3
...,...,...,...,...,...
18916,1.0,1.0,0.0,1.000000,116
18925,0.0,0.0,0.0,0.000000,41
18959,1.0,0.0,2.0,0.000000,55
18971,6.0,3.0,6.0,0.333333,283


In [167]:
rf=RandomForestClassifier()

In [168]:
parametrs={'criterion': ['entropy','gini'],
 'n_estimators': range(10, 50, 10),
 'max_depth': range(1, 12, 2),
 'min_samples_leaf': range(1, 7),
 'min_samples_split': (2, 9, 2)}

In [169]:
search=GridSearchCV(rf,parametrs,cv=3,n_jobs=-1)

In [170]:
search.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 12, 2),
                         'min_samples_leaf': range(1, 7),
                         'min_samples_split': (2, 9, 2),
                         'n_estimators': range(10, 50, 10)})

In [171]:
best_rf=search.best_estimator_

In [172]:
best_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=11, min_samples_leaf=3, n_estimators=30)

In [173]:
X_test=pd.DataFrame({'user_id':events_data_test['user_id'].unique()})

In [174]:
steps_tried=submission_data_test.groupby('user_id').step_id.nunique().to_frame().reset_index().rename({'step_id':"steps_tried"},axis=1)

In [175]:
X_test=X_test.merge(steps_tried[['user_id','steps_tried']],how='outer').fillna(0)

In [176]:
X_test

,user_id,steps_tried
0,24417,0.0
1,999,0.0
2,26244,0.0
3,14192,0.0
4,12684,0.0
...,...,...
6179,7315,0.0
6180,24300,0.0
6181,820,0.0
6182,3649,0.0


In [177]:
users_scores=submission_data_test.pivot_table(index="user_id",columns="submission_status",values="step_id",aggfunc="count",fill_value=0).reset_index()

In [178]:
X_test=X_test.merge(users_scores[['user_id','correct','wrong']],how='outer').fillna(0)

In [179]:
X_test['cor_score']=X_test.correct/(X_test.correct+X_train.wrong)

In [180]:
X_test['cor_score']=X_test['cor_score'].fillna(0)

In [181]:
X_test

,user_id,steps_tried,correct,wrong,cor_score
0,24417,0.0,0.0,0.0,0.0
1,999,0.0,0.0,0.0,0.0
2,26244,0.0,0.0,0.0,0.0
3,14192,0.0,0.0,0.0,0.0
4,12684,0.0,0.0,0.0,0.0
...,...,...,...,...,...
6179,7315,0.0,0.0,0.0,0.0
6180,24300,0.0,0.0,0.0,0.0
6181,820,0.0,0.0,0.0,0.0
6182,3649,0.0,0.0,0.0,0.0


In [182]:
users_events=events_data_test.pivot_table(index="user_id",columns="action",values="step_id",aggfunc="count",fill_value=0).reset_index()

In [183]:
X_test=X_test.merge(users_events[['user_id','viewed']])

In [184]:
X_test

,user_id,steps_tried,correct,wrong,cor_score,viewed
0,24417,0.0,0.0,0.0,0.0,65
1,999,0.0,0.0,0.0,0.0,8
2,26244,0.0,0.0,0.0,0.0,1
3,14192,0.0,0.0,0.0,0.0,22
4,12684,0.0,0.0,0.0,0.0,336
...,...,...,...,...,...,...
6179,7315,0.0,0.0,0.0,0.0,1
6180,24300,0.0,0.0,0.0,0.0,1
6181,820,0.0,0.0,0.0,0.0,2
6182,3649,0.0,0.0,0.0,0.0,1


In [185]:
predict=best_rf.predict_proba(X_test.drop('user_id',axis=1))

In [186]:
predict_data=pd.DataFrame({'user_id':X_test.user_id,'is_gone':predict[:,1]})

In [190]:
predict_data.to_csv('ans1.csv',index=False)

In [189]:
predict_data

,user_id,is_gone
0,24417,0.052708
1,999,0.003951
2,26244,0.000868
3,14192,0.000212
4,12684,0.177960
...,...,...
6179,7315,0.000868
6180,24300,0.000868
6181,820,0.003044
6182,3649,0.000868
